# 機械学習をPythonで実践する-17　　～ 欠損値対応-2 ～
様々な欠損対応手法を適用した場合の機械学習モデルの精度を比較する。  
ここではKaggleのpenguin datasetを例として下記それぞれのケースで精度を比較する。  
目的変数は'species'（ペンギンの種別）とする。  
[penguin dataset](https://www.kaggle.com/code/parulpandey/penguin-dataset-the-new-iris/input?select=penguins_size.csv)  
* 欠損値を落としたケース
* 欠損値を新しいカテゴリとしたケース（数値カラムは中央値で代入する）
* 量的変数の欠損値をkNNで予測したケース（Classifierまでやるのは大変なので、カテゴリ変数は最頻値で代入）

学習モデルにはLightGBMを用いる。  
※LightGBMでは標準化、欠損値対応は必須ではないが、練習のためやる。対応しない場合との精度も比較する。



In [37]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
# accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier


%matplotlib inline
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ○ データセット読み込み、値チェック

In [40]:
# データセットのカラム名やデータ型は分かっているので、dtypesを指定しておく。
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

In [41]:
df.head()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
str,str,f32,f32,f32,f32,str
"""Adelie""","""Torgersen""",39.099998,18.700001,181.0,3750.0,"""MALE"""
"""Adelie""","""Torgersen""",39.5,17.4,186.0,3800.0,"""FEMALE"""
"""Adelie""","""Torgersen""",40.299999,18.0,195.0,3250.0,"""FEMALE"""
"""Adelie""","""Torgersen""",null,null,null,null,null
"""Adelie""","""Torgersen""",36.700001,19.299999,193.0,3450.0,"""FEMALE"""


In [42]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,2,2,2,2,10


 "NA"は欠損値として扱う。  
 他にも欠損値のような値が入っていないかチェックする。  
 f32のdtypeのカラムには少なくとも文字列で表現される欠損値はないと考えられる。

In [43]:
df.describe()

describe,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
str,str,str,f64,f64,f64,f64,str
"""count""","""344""","""344""",344.0,344.0,344.0,344.0,"""344"""
"""null_count""","""0""","""0""",2.0,2.0,2.0,2.0,"""10"""
"""mean""",null,null,43.921928,17.151169,200.915207,4201.754395,null
"""std""",null,null,5.459584,1.974793,14.061713,801.954468,null
"""min""","""Adelie""","""Biscoe""",32.099998,13.1,172.0,2700.0,"""."""
"""max""","""Gentoo""","""Torgersen""",59.599998,21.5,231.0,6300.0,"""MALE"""
"""median""",null,null,44.450001,17.299999,197.0,4050.0,null


culmen_length,depth,flipper_length,body_mass_gの最小値・最大値に特異な値(0,9999など)は入っていないため、  
null以外の欠損値候補はないと思われる。  
一方、sexのminに"."が入っているため、詳しく見てみる。

In [44]:
df['sex'].value_counts()

sex,counts
str,u32
"""FEMALE""",165
"""MALE""",168
""".""",1
null,10


"."が欠損値なのかそれ以外の意味を持つのか判断できない。    
わかりやすいように一旦"others"で置き換え、別のクラスの扱いとする。  

In [52]:
# sexカラムの'.'を'others'で置き換え
df = df.with_columns(
    pl.when(pl.col('sex') == '.').then('others').otherwise(pl.col('sex')).alias('sex')
)

In [53]:
df['sex'].value_counts()

sex,counts
str,u32
"""FEMALE""",165
null,10
"""MALE""",168
"""others""",1


残りのカテゴリカラムについても'Unknown','N/A'などないかチェック。

In [55]:
df['species'].value_counts()

species,counts
str,u32
"""Adelie""",152
"""Gentoo""",124
"""Chinstrap""",68


In [57]:
df['island'].value_counts()

island,counts
str,u32
"""Dream""",124
"""Biscoe""",168
"""Torgersen""",52


species,islanに重複が疑われるクラスはない。  
目的変数speciesにおいて、Chinstrapは他のクラスよりも半分以下の数になっているため、  
CVをする際はStratified kFoldを使うのが良いかも。  